In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from hmftpy.operators import periodic_hamiltonian
from tqdm import tqdm

from hmftpy.plaquettes.triangular import plaq3, plaq12, plaq19

# Hamiltonian
Here, we consider the Hamiltonian
\begin{align*}
    H = &\sum_{\langle i, j\rangle} \vec D_{ij} \cdot \left(\vec \sigma_i \times\vec \sigma_j\right)
        + \sum_{\langle i, j\rangle} J_{ij} \vec \sigma_i \cdot \vec \sigma_j
        + B \sum_i \sigma_i^z.
\end{align*}
where $\vec D_{ij}=D\hat z$ for nearest-neighbor bonds $\langle i,j \rangle$ and zero elsewise, and similarly $J_{ij}=J$ for $\langle i, j\rangle$ nearest-neighbors. For now, I will set $B=0$.


Nvmnd it's a JK! The paper I'm duplicating ([this one](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.103.L060404)) orients $\vec D_{ij}$ to be perpendicular to the direction (in the plane of the lattice) of the bond between sites $i$ and $j$.

The triangular lattice has primitive vectors
\begin{equation}
\vec a_1 = \begin{pmatrix} 1 \\ 0 \end{pmatrix},
\quad
\vec a_2 = \begin{pmatrix} \frac{1}{2} \\ \frac{\sqrt{3}}{2} \end{pmatrix}.
\end{equation}
Each site on the lattice is connected to its six nearest neighbors by a linear combination of these vectors.

In my code, I refer to bonds aligned with $a_1$ as $x$ bonds, $a_2$ as $z$ bonds, and $a_2-a_1$ as $y$ bonds.
The perpendicular directions to these (with a particular sign choice so all perpendicular arrows point out of the
upright triangles) are
\begin{align*}
\hat x_\perp =& \begin{pmatrix} 0 \\ -1 \end{pmatrix}\\
\hat y_\perp =& \begin{pmatrix} -\frac{\sqrt 3}{2} \\ \frac{1}{2} \end{pmatrix} \\
\hat z_\perp =& \begin{pmatrix} \frac{\sqrt 3}{2} \\ \frac{1}{2} \end{pmatrix}
\end{align*}

In [ ]:
u = np.array([1,0])
up = np.array([0,-1])

v = np.array([-0.5, np.sqrt(3)/2])
vp = np.array([np.sqrt(3)/2, 0.5])

w = np.array([0.5, np.sqrt(3)/2])
wp = np.array([-np.sqrt(3)/2, 0.5])

x = np.array([1,0])
y = np.array([0,1])

In [ ]:
# general code
def s_interactions(h, J, D):
    interactions = {'local': {'z': -h},
                    'x_bonds': {'yz': D*np.vdot(x, up), 
                                'zy': -D*np.vdot(x, up),
                                'zx': D*np.vdot(y, up),
                                'xz': -D*np.vdot(y, up),
                                'xx': J, 'yy': J, 'zz': J},
                    'y_bonds': {'yz': D*np.vdot(x, vp), 
                                'zy': -D*np.vdot(x, vp),
                                'zx': D*np.vdot(y, vp),
                                'xz': -D*np.vdot(y, vp),
                                'xx': J, 'yy': J, 'zz': J},
                    'z_bonds': {'yz': D*np.vdot(x, wp), 
                                'zy': -D*np.vdot(x, wp),
                                'zx': D*np.vdot(y, wp),
                                'xz': -D*np.vdot(y, wp),
                                'xx': J, 'yy': J, 'zz': J}}
    return interactions


def Mz_op(L, basis):
    return quantum_operator({'static': [['z', [[1./L, i] for i in range(L)]]]}, 
                            basis=basis)

def chi_op(tris, basis):
    l = len(tris)
    n = 1./l
    return quantum_operator({'static': [['xyz', [[n,t[0],t[1],t[2]] for t in tris]],
                                        ['xzy', [[-n,t[0],t[1],t[2]] for t in tris]],
                                        ['yzx', [[n,t[0],t[1],t[2]] for t in tris]],
                                        ['yxz', [[-n,t[0],t[1],t[2]] for t in tris]],
                                        ['zxy', [[n,t[0],t[1],t[2]] for t in tris]],
                                        ['zyx', [[-n,t[0],t[1],t[2]] for t in tris]]]}, 
                            basis=basis)

# 3-site cluster

First, let's just write the vectors for the bonds on our 3-site cluster below:

            <0>-<1>-<2>
            / \ / \ / \
          <1>- 2 -<0>-<1>
          / \ / \ / \ /
        <2>- 0 - 1 -<2>-
          \ / \ / \ / \
          <1>-<2>-<0>-<1>
          
The ones within the cluster are:
\begin{align}
\vec r_{01}^0 = \vec a_1, \quad
\vec r_{12}^0 = \vec a_2 - \vec a_1, \quad
\vec r_{20}^0 = -\vec a_2.
\end{align}
Each pair of nearest-neighbors in the lattice is connected by one of these vectors! I'm going to give these directions names:
\begin{align}
\hat u =& \vec a_1\\
\hat v =& \vec a_2-\vec a_1\\
\hat w =& -\vec a_2
\end{align}

I'm going to make a list of the bonds in our three site cluster and a corresponding list of the directions perpendicular to those bonds:

In [ ]:
steps = 20
hs = np.linspace(0, 3, steps)
D = 1
J = -0.5
basis = spin_basis_1d(3, pauli=0)
es = np.zeros(steps)
vs = np.zeros((steps, basis.Ns), dtype=np.complex128)


In [ ]:
for i, h in enumerate(tqdm(hs)):
    interactions = s_interactions(h, J, D)
    H = periodic_hamiltonian(plaq3, interactions, basis)
    e, v = H.eigh()
    es[i] = e[0]
    vs[i] = v[:,0]

In [ ]:
mz = Mz_op(3, basis)
tris = [[0,1,2], [1,2,0], [2,0,1]]
chi = chi_op(tris, basis)

In [ ]:
plt.figure(figsize=(4,3), dpi=300)
plt.plot(hs, es/3, label=r'$e$')
plt.plot(hs, [mz.expt_value(vs[i]).real for i in range(steps)], label=r'$M_z$')
plt.plot(hs, [chi.expt_value(vs[i]).real for i in range(steps)], label=r'$\chi$')
plt.xlabel('h')
plt.legend()
plt.show()

Well, that's boring! How about we move to a larger cluster?

I know that the 12-site cluster below is usefull because it preserves both rotational and sublattice symmetries, and it's small enough to be easy to work with.

                <4>-<5>
                / \ / \
          <9>-<0>-<1>-<2>-<7>
            \ / \ / \ / \ /
        <5>-<6>-10 -11 -<3>-<4>
          \ / \ / \ / \ / \ /
      <1>-<2>- 7 - 8 - 9 -<0>-<1>
        \ / \ / \ / \ / \ / \ /
    <10-<11- 3 - 4 - 5 - 6 -<10-<11
      \ / \ / \ / \ / \ / \ / \ /
      <8>-<9>- 0 - 1 - 2 -<7>-<8>
          / \ / \ / \ / \ / \
        <5>-<6>-<10-<11-<3>-<4>
              \ / \ / \ /
              <7>-<8>-<9>
              


Now, our lists of neighbors are larger! Gotta love it!

In [ ]:
steps = 20
hs = np.linspace(0, 1, steps)
D = 1
J = -0.5
basis = spin_basis_1d(12, pauli=0)
es = np.zeros(steps)
vs = np.zeros((steps, basis.Ns), dtype=np.complex128)

In [ ]:
for i, h in enumerate(tqdm(hs)):
    interactions = s_interactions(h, J, D)
    H = periodic_hamiltonian(plaq12, interactions, basis)
    e, v = H.eigsh(k=1, which='SA')
    es[i] = e[0]
    vs[i] = v[:,0]

In [ ]:
mz = Mz_op(12, basis)
# tris = [[0,1,4], [1,2,5], [3,4,7], [4,5,8], [5,6,9], [7,8,10], [8,9,11],
#         [2,7,6], [6,10,0], [9,0,3], [11,3,2], [10,11,1]]
tris = [[0,4,3]]
c = 12/np.pi
chi = quantum_operator({'static': [['xyz', [[c,t[0],t[1],t[2]] for t in tris]],
                                   ['xzy', [[-c,t[0],t[1],t[2]] for t in tris]],
                                   ['yzx', [[c,t[0],t[1],t[2]] for t in tris]],
                                   ['yxz', [[-c,t[0],t[1],t[2]] for t in tris]],
                                   ['zxy', [[c,t[0],t[1],t[2]] for t in tris]],
                                   ['zyx', [[-c,t[0],t[1],t[2]] for t in tris]]]}, basis=basis)

In [ ]:
plt.figure(figsize=(4,3), dpi=300)
plt.plot(hs, es/12, label=r'$e$', marker='.')
plt.plot(hs, [mz.expt_value(vs[i]).real for i in range(steps)], label=r'$M_z$', marker='.')
plt.plot(hs, [chi.expt_value(vs[i]).real for i in range(steps)], label=r'$\chi$', marker='.')
plt.xlabel('h')
plt.legend()
plt.show()

# 19 site!
          
         <0> <1> <2>
           \   \   \  
      <11> 16 -17 -18 -<3>
         \ / \ / \ / \  
     <6> 12 -13 -14 -15 -<0>
       \ / \ / \ / \ / \   
        7 - 8 - 9 -10 -11 -<16
       / \ / \ / \ / \ /  
    <18>  3 - 4 - 5 - 6 -<12
         / \ / \ / \ /  
      <15>  0 - 1 - 2 -<7
           /   /   /
        <11 <16 <17 
        
It's this one!

In [ ]:
steps = 15
hs = np.linspace(0, 1, steps)
D = 1
J = -0.5
basis = spin_basis_1d(19, pauli=0)
es = np.zeros(steps)
vs = np.zeros((steps, basis.Ns), dtype=np.complex128)

In [ ]:
for i, h in enumerate(tqdm(hs)):
    interactions = s_interactions(h, J, D)
    H = periodic_hamiltonian(plaq19, interactions, basis)
    e, v = H.eigsh(k=1, which='SA', tol=10**-8)
    es[i] = e[0]
    vs[i] = v[:,0]

In [ ]:
mz = Mz_op(19, basis)
tris = [[1,2,5]]
# tries = [[3,0,4], [1,2,5]]
# tris = [[0,1,4], [1,2,5], [3,4,8], [4,5,9], [5,6,10], 
#         [7,8,12], [8,9,13], [9,10,14], [10,11,15], [12,13,16], [13,14,17], [14,15,18],
#         [2,7,6], [6,12,11], [11,16,0], [15,0,3], [17,18,2], [16,17,1], [18,3,7]] 
c = 19/np.pi
# c = 1/np.pi
chi = quantum_operator({'static': [['xyz', [[c,t[0],t[1],t[2]] for t in tris]],
                                   ['xzy', [[-c,t[0],t[1],t[2]] for t in tris]],
                                   ['yzx', [[c,t[0],t[1],t[2]] for t in tris]],
                                   ['yxz', [[-c,t[0],t[1],t[2]] for t in tris]],
                                   ['zxy', [[c,t[0],t[1],t[2]] for t in tris]],
                                   ['zyx', [[-c,t[0],t[1],t[2]] for t in tris]]]}, basis=basis)

In [ ]:
plt.figure(figsize=(4,3), dpi=300)
# plt.plot(hs, es/12, label=r'$e$', marker='.')
plt.plot(hs, [mz.expt_value(vs[i]).real for i in range(steps)], label=r'$M_z$', marker='.')
plt.xlabel('h')
plt.legend()
plt.show()

plt.figure(figsize=(4,3), dpi=300)
plt.plot(hs, [chi.expt_value(vs[i]).real for i in range(steps)], label=r'$\chi$', marker='.')
plt.xlabel('h')
plt.legend()
plt.show()